# DataFrameIt - Rate Limiting

Este notebook demonstra como configurar o `rate_limit_delay` para evitar atingir limites de requisições das APIs de LLM.

**Conceitos demonstrados:**
- Rate limiting proativo
- Configuração para diferentes providers (Gemini, OpenAI, Anthropic)
- Cálculo do delay ideal
- Dupla proteção: Rate Limiting + Retry

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/rate_limiting.ipynb)

## 1. Instalação

In [ ]:
!pip install -q dataframeit[google]

## 2. Configuração da API Key

In [ ]:
import os

try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import pandas as pd
from dataframeit import dataframeit

## 4. Definir Modelo e Template

In [ ]:
class AnaliseTexto(BaseModel):
    """Modelo para estruturar análise de textos."""
    sentimento: Literal['positivo', 'negativo', 'neutro'] = Field(
        description="Sentimento geral do texto"
    )
    tema_principal: str = Field(
        description="Tema ou assunto principal do texto"
    )
    resumo: str = Field(
        description="Resumo breve do texto em até 50 palavras"
    )


TEMPLATE = """
Você é um analista de texto especializado.

Analise o texto a seguir e extraia:
1. O sentimento geral (positivo, negativo ou neutro)
2. O tema principal abordado
3. Um resumo breve (máximo 50 palavras)

Texto a analisar:
{texto}
"""

## 5. Criar Dataset de Exemplo

In [ ]:
textos = [
    "A nova política de saúde pública trouxe melhorias significativas no atendimento.",
    "O sistema de transporte continua enfrentando problemas graves de superlotação.",
    "Investimentos em educação foram anunciados pelo governo federal.",
    "A economia apresentou crescimento moderado no último trimestre.",
    "Manifestantes protestam contra aumento de impostos nas principais cidades.",
    "Novo programa social visa reduzir desigualdade no acesso à tecnologia.",
    "Empresas de tecnologia anunciam cortes de pessoal devido à crise.",
    "Cientistas descobrem nova espécie de planta na Amazônia.",
    "Mudanças climáticas afetam produção agrícola em várias regiões.",
    "Campanha de vacinação atinge meta estabelecida pelo ministério.",
]

df = pd.DataFrame({'texto': textos})
print(f"Dataset com {len(df)} textos para processar")
df.head()

## 6. Por que usar Rate Limiting?

- **Prevenir erros**: Evita atingir limites de requisições antes de acontecer
- **Eficiência**: Reduz desperdício de retries em datasets grandes
- **Economia**: Algumas APIs cobram por tentativa, mesmo as que falham
- **Complementar ao Retry**: O rate limiting PREVINE erros, o retry TRATA erros

### Quando usar?
- Processar datasets grandes (> 100 linhas)
- Conhecer os limites da API (ex: 60 req/min)
- Fazer processamento em lote
- Quiser economizar retries para erros reais

## 7. Exemplo: Google Gemini com Rate Limiting

In [ ]:
print("Google Gemini com rate limiting")
print("API Limit: 60 requisições/minuto (free tier)")
print("Rate Limit Delay: 1.0 segundo entre requisições\n")

resultado = dataframeit(
    df,
    AnaliseTexto,
    TEMPLATE,
    provider='google_genai',
    rate_limit_delay=1.0  # 60 req/min = 1 req/segundo
)

print(f"\nProcessamento concluído! Total de linhas: {len(resultado)}")

## 8. Visualizar Resultados

In [ ]:
resultado[['texto', 'sentimento', 'tema_principal']].head()

## 9. Configurações para Outros Providers

### OpenAI GPT-4 (500 req/min)
```python
resultado = dataframeit(
    df,
    AnaliseTexto,
    TEMPLATE,
    provider='openai',
    model='gpt-4o-mini',
    rate_limit_delay=0.15  # ~400 req/min (margem de segurança)
)
```

### Anthropic Claude (50 req/min)
```python
resultado = dataframeit(
    df,
    AnaliseTexto,
    TEMPLATE,
    provider='anthropic',
    model='claude-3-5-sonnet-20241022',
    rate_limit_delay=1.2  # 50 req/min
)
```

## 10. Cálculo do Delay Ideal

Fórmula:
```
rate_limit_delay = 60 / limite_de_requisições_por_minuto
```

Exemplos:
- 60 req/min  → delay = 60/60  = 1.0 segundo
- 500 req/min → delay = 60/500 = 0.12 segundos
- 50 req/min  → delay = 60/50  = 1.2 segundos

In [ ]:
def calcular_delay_ideal(req_por_minuto: int, margem_seguranca: float = 0.9):
    """
    Calcula o delay ideal baseado no limite de requisições da API.

    Args:
        req_por_minuto: Limite de requisições por minuto da API
        margem_seguranca: Fator de segurança (0.9 = usar 90% do limite)

    Returns:
        Delay em segundos entre requisições
    """
    req_ajustado = req_por_minuto * margem_seguranca
    delay = 60 / req_ajustado

    print(f"Cálculo de Rate Limit:")
    print(f"  Limite da API: {req_por_minuto} req/min")
    print(f"  Com margem de segurança ({int(margem_seguranca*100)}%): {req_ajustado:.0f} req/min")
    print(f"  Delay recomendado: {delay:.2f} segundos")

    return delay

# Exemplo de uso
delay = calcular_delay_ideal(req_por_minuto=60, margem_seguranca=0.9)

## 11. Dupla Proteção: Rate Limiting + Retry

Para máxima estabilidade, combine rate limiting com retry.

In [ ]:
print("Processando com dupla proteção...")
print("  Rate Limiting: Previne erros de rate limit")
print("  Retry: Trata erros inesperados com backoff exponencial\n")

resultado_protegido = dataframeit(
    df,
    AnaliseTexto,
    TEMPLATE,
    # Rate limiting proativo
    rate_limit_delay=1.0,

    # Retry reativo
    max_retries=3,
    base_delay=2.0,
    max_delay=30.0
)

print(f"\nProcessamento concluído com dupla proteção!")

# Verificar se houve erros
erros = resultado_protegido[resultado_protegido['_dataframeit_status'] == 'error']
sucesso = resultado_protegido[resultado_protegido['_dataframeit_status'] == 'processed']

print(f"Sucessos: {len(sucesso)}")
print(f"Erros: {len(erros)}")

## 12. Processamento Paralelo com Rate Limiting

Para máxima estabilidade em processamento paralelo, combine `parallel_requests` com `rate_limit_delay`.

```python
# 5 workers paralelos, com 0.5s entre cada requisição por worker
resultado = dataframeit(
    df,
    AnaliseTexto,
    TEMPLATE,
    parallel_requests=5,
    rate_limit_delay=0.5     # Delay adicional por requisição
)
```

### Dicas de Configuração

| Cenário | Configuração Recomendada |
|---------|-------------------------|
| Dataset pequeno (< 50 linhas) | `parallel_requests=1` (padrão) |
| Dataset médio (50-500 linhas) | `parallel_requests=3` a `5` |
| Dataset grande (> 500 linhas) | `parallel_requests=5` a `10` |
| API com limite baixo | `parallel_requests=2` + `rate_limit_delay=1.0` |
| Tier pago com limites altos | `parallel_requests=10` ou mais |

## Resumo

- **Rate Limiting** previne erros proativamente
- **Retry** trata erros reativamente
- Use ambos para máxima estabilidade
- Calcule o delay ideal baseado nos limites da sua API
- Para datasets grandes, combine com `parallel_requests`

---

## Próximos Passos

- [02_error_handling.ipynb](02_error_handling.ipynb) - Tratamento de erros
- [03_resume.ipynb](03_resume.ipynb) - Processamento incremental